In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats as stats
import warnings
warnings.simplefilter("ignore")

In [2]:
data = pd.read_csv('data.csv', dtype={'id_prev':str, 'id':str})
data

,year,project_name,division,id_prev,id,type_institution,institution,start_date,end_date,K-NSCC,...,royalty,sales,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm,pat_nm,start_year,end_year
0,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415143072,1415149752,출연연구소,한국전기연구원,2013-08-01,2017-01-31,에너지/자원,...,NaN,NaN,0.053476,0.053476,NaN,NaN,NaN,1.0,2013,2017.0
1,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415141390,1415146761,대기업,에스케이이노베이션(주),2013-06-01,2017-05-31,에너지/자원,...,NaN,66000000.0,NaN,NaN,NaN,NaN,0.372671,NaN,2013,2017.0
2,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415143359,1415148130,출연연구소,한국기계연구원,2014-12-01,2019-09-30,원자력,...,NaN,NaN,NaN,NaN,0.045455,NaN,NaN,NaN,2014,2019.0
3,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415141388,1415146758,대기업,(주)엘지유플러스,2013-06-01,2017-05-31,에너지/자원,...,NaN,532203426.0,NaN,NaN,NaN,NaN,5.054164,NaN,2013,2017.0
4,2016.0,전력표준화및인증지원사업,일반연구개발사업,NaN,1415150219,기타,한국광기술원,2016-06-01,2021-05-31,전기/전자,...,NaN,NaN,0.200000,0.000000,NaN,NaN,NaN,0.0,2016,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98220,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711099086,대학,한양대학교(ERICA캠퍼스),2019-09-27,2022-12-31,재료,...,NaN,NaN,2.061856,0.000000,NaN,NaN,NaN,0.0,2019,2022.0
98221,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711098190,대학,포항공과대학교,2019-07-15,2025-06-30,재료,...,NaN,NaN,0.400000,0.000000,0.200000,NaN,NaN,0.0,2019,2025.0
98222,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711077275,1711093328,기타,전자부품연구원,2018-07-16,2024-07-15,전기/전자,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2018,2024.0
98223,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711076433,1711083081,대학,고려대학교,2018-07-16,2024-07-15,재료,...,NaN,NaN,3.428571,0.000000,5.142857,NaN,NaN,0.0,2018,2024.0


In [3]:
data.rename(columns = {'K-NSCC':'technology_type'}, inplace=True)

# Remove outliers

In [4]:
condition = data[data['id']=='1711037839'].index
data.drop(condition, inplace=True)
data.reset_index()

,index,year,project_name,division,id_prev,id,type_institution,institution,start_date,end_date,...,royalty,sales,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm,pat_nm,start_year,end_year
0,0,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415143072,1415149752,출연연구소,한국전기연구원,2013-08-01,2017-01-31,...,NaN,NaN,0.053476,0.053476,NaN,NaN,NaN,1.0,2013,2017.0
1,1,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415141390,1415146761,대기업,에스케이이노베이션(주),2013-06-01,2017-05-31,...,NaN,66000000.0,NaN,NaN,NaN,NaN,0.372671,NaN,2013,2017.0
2,2,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415143359,1415148130,출연연구소,한국기계연구원,2014-12-01,2019-09-30,...,NaN,NaN,NaN,NaN,0.045455,NaN,NaN,NaN,2014,2019.0
3,3,2016.0,전력표준화및인증지원사업,일반연구개발사업,1415141388,1415146758,대기업,(주)엘지유플러스,2013-06-01,2017-05-31,...,NaN,532203426.0,NaN,NaN,NaN,NaN,5.054164,NaN,2013,2017.0
4,4,2016.0,전력표준화및인증지원사업,일반연구개발사업,NaN,1415150219,기타,한국광기술원,2016-06-01,2021-05-31,...,NaN,NaN,0.200000,0.000000,NaN,NaN,NaN,0.0,2016,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98219,98220,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711099086,대학,한양대학교(ERICA캠퍼스),2019-09-27,2022-12-31,...,NaN,NaN,2.061856,0.000000,NaN,NaN,NaN,0.0,2019,2022.0
98220,98221,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711098190,대학,포항공과대학교,2019-07-15,2025-06-30,...,NaN,NaN,0.400000,0.000000,0.200000,NaN,NaN,0.0,2019,2025.0
98221,98222,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711077275,1711093328,기타,전자부품연구원,2018-07-16,2024-07-15,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2018,2024.0
98222,98223,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711076433,1711083081,대학,고려대학교,2018-07-16,2024-07-15,...,NaN,NaN,3.428571,0.000000,5.142857,NaN,NaN,0.0,2018,2024.0


# Performance analysis by research phase and technology type

In [5]:
data.groupby('phase').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='count'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='count'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='count'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='count'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='count')}).reset_index()

,phase,application,registration,publication,royalty,sales
0,개발연구,21079,21079,7623,4932,11770
1,기초연구,19169,19169,42053,1301,1864
2,기타,3408,3408,2754,885,1060
3,응용연구,8030,8030,6818,1372,3620


In [6]:
data.groupby('phase').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='mean'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='mean'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='mean'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='mean'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='mean')}).reset_index()

,phase,application,registration,publication,royalty,sales
0,개발연구,1.303512,0.358366,1.879797,0.109946,4.992919
1,기초연구,2.440500,0.821136,4.413865,0.234804,5.099854
2,기타,3.140346,1.187532,11.496783,0.367318,4.267333
3,응용연구,1.558375,0.518171,2.705945,0.180331,2.892496


In [7]:
data.groupby('phase').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='std'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='std'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='std'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='std'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='std')}).reset_index()

,phase,application,registration,publication,royalty,sales
0,개발연구,3.302886,1.146493,18.434659,0.426095,28.016870
1,기초연구,3.750896,2.207938,7.328853,0.744158,122.774438
2,기타,6.208400,4.101529,15.348610,1.568585,37.572219
3,응용연구,3.474754,1.702754,8.576187,0.716195,97.475138


In [8]:
data.groupby('technology_type').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='mean'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='mean'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='mean'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='mean'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='mean')}).reset_index()

,technology_type,application,registration,publication,royalty,sales
0,건설/교통,1.278380,0.484124,3.070614,0.126313,5.761475
1,경제/경영,2.759876,1.098387,5.144426,0.166211,6.854464
2,과학기술과 인문사회,0.732630,0.186949,2.941523,0.076527,0.108134
3,교육,1.394501,0.254121,2.768107,0.105959,3.103259
4,기계,1.628408,0.587113,3.817229,0.128025,6.913307
5,기타,0.404955,0.220127,NaN,0.037366,NaN
6,농림수산식품,2.330408,0.737345,3.380924,0.137711,3.437912
7,뇌과학,1.646686,0.511348,3.413002,0.083723,1.148388
8,문학,3.393588,3.393588,0.670780,NaN,NaN
9,문화/예술/체육,1.398400,0.399697,3.419902,0.212478,1.536401


In [9]:
dataset = data[(data['technology_type']=='보건의료') | (data['technology_type']=='농림수산식품') | (data['technology_type']=='기계') | (data['technology_type']=='생명과학') | (data['technology_type']=='정보/통신')]
dataset.groupby('technology_type').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='count'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='count'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='count'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='count'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='count')}).reset_index()

,technology_type,application,registration,publication,royalty,sales
0,기계,5886,5886,3577,1188,2733
1,농림수산식품,5014,5014,5224,1318,5322
2,보건의료,7057,7057,12023,692,1052
3,생명과학,4015,4015,6939,468,710
4,정보/통신,6007,6007,3936,1428,2063


In [10]:
dataset = data[(data['technology_type']=='보건의료') | (data['technology_type']=='농림수산식품') | (data['technology_type']=='기계') | (data['technology_type']=='생명과학') | (data['technology_type']=='정보/통신')]
dataset.groupby('technology_type').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='mean'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='mean'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='mean'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='mean'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='mean')}).reset_index()

,technology_type,application,registration,publication,royalty,sales
0,기계,1.628408,0.587113,3.817229,0.128025,6.913307
1,농림수산식품,2.330408,0.737345,3.380924,0.137711,3.437912
2,보건의료,1.939694,0.526282,3.899717,0.337420,2.363114
3,생명과학,2.097852,0.614932,3.684368,0.243415,2.286289
4,정보/통신,2.112465,0.810795,3.538069,0.127913,3.511113


In [11]:
dataset = data[(data['technology_type']=='보건의료') | (data['technology_type']=='농림수산식품') | (data['technology_type']=='기계') | (data['technology_type']=='생명과학') | (data['technology_type']=='정보/통신')]
dataset.groupby('technology_type').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='std'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='std'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='std'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='std'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='std')}).reset_index()

,technology_type,application,registration,publication,royalty,sales
0,기계,4.046538,2.349062,7.321359,0.276114,31.428590
1,농림수산식품,3.935921,1.825534,5.973089,0.332928,106.166933
2,보건의료,2.876343,1.377645,16.689060,1.195947,12.719223
3,생명과학,2.956237,1.439836,5.502146,0.590161,13.255019
4,정보/통신,4.545510,3.510786,5.670774,0.386836,33.100400


In [12]:
data.groupby('6T').agg(**{
    'application':pd.NamedAgg(column='app_nm', aggfunc='mean'),
    'registration':pd.NamedAgg(column='reg_nm', aggfunc='mean'),
    'publication':pd.NamedAgg(column='pub_nm', aggfunc='mean'),
    'royalty':pd.NamedAgg(column='royalty_nm', aggfunc='mean'),
    'sales': pd.NamedAgg(column='sales_nm', aggfunc='mean')}).reset_index()

,6T,application,registration,publication,royalty,sales
0,BT(생명공학기술),2.094499,0.611748,3.729692,0.210567,3.255381
1,CT(문화기술),1.327415,0.401812,2.432887,0.136692,1.621294
2,ET(환경기술),1.317395,0.428387,3.238253,0.121046,5.628202
3,IT(정보기술),1.988188,0.708761,4.046196,0.122899,4.022064
4,NT(나노기술),2.478316,0.791472,5.544129,0.351814,4.978672
5,ST(우주항공기술),1.481600,0.590146,4.185482,0.217752,9.056674
6,위의 미래유망신기술(6T) 103개 세분류에 속하지 않는 기타 연구,1.617187,0.560918,5.659154,0.145178,7.239113


# Normality test

In [13]:
# applied patent-research phase

phase_list = data['phase'].unique()
result = {}
for i in phase_list:
    app_data = data[data['app_nm'].notnull()]
    app_normality = stats.shapiro(app_data.loc[app_data['phase'] == i, 'app_nm'])
    result[i] = [app_normality.statistic, app_normality.pvalue, app_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
기타,0.481684,0.0,False
개발연구,0.222373,0.0,False
응용연구,0.31759,0.0,False
기초연구,0.496643,0.0,False


In [14]:
# applied patent-tehcnology type

technology_type_list = data['technology_type'].unique()
result={}

for i in technology_type_list:
    if len(app_data[app_data['technology_type'] == i]) > 3:
        app_data = data[data['app_nm'].notnull()]
        app_normality = stats.shapiro(app_data.loc[app_data['technology_type'] == i, 'app_nm'])
        result[i] = [app_normality.statistic, app_normality.pvalue, app_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
에너지/자원,0.259995,0.0,False
원자력,0.352962,0.0,False
전기/전자,0.252919,0.0,False
보건의료,0.549192,0.0,False
생명과학,0.582563,0.0,False
뇌과학,0.559175,0.0,False
화학,0.661796,0.0,False
화공,0.409235,0.0,False
재료,0.407769,0.0,False
심리,0.723434,0.000448,False


In [15]:
# registered patent-research phase

phase_list = data['phase'].unique()
result = {}
for i in phase_list:
    reg_data = data[data['reg_nm'].notnull()]
    reg_normality = stats.shapiro(reg_data.loc[reg_data['phase'] == i, 'reg_nm'])
    result[i] = [reg_normality.statistic, reg_normality.pvalue, reg_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
기타,0.272496,0.0,False
개발연구,0.279778,0.0,False
응용연구,0.290218,0.0,False
기초연구,0.340333,0.0,False


In [16]:
# registered patent-technology type

technology_type_list = data['technology_type'].unique()
result={}

for i in technology_type_list:
    if len(reg_data[reg_data['technology_type'] == i]) > 3:
        reg_data = data[data['reg_nm'].notnull()]
        reg_normality = stats.shapiro(reg_data.loc[reg_data['technology_type'] == i, 'reg_nm'])
        result[i] = [reg_normality.statistic, reg_normality.pvalue, reg_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
에너지/자원,0.294613,0.0,False
원자력,0.22268,0.0,False
전기/전자,0.380673,0.0,False
보건의료,0.393751,0.0,False
생명과학,0.463382,0.0,False
뇌과학,0.448589,0.0,False
화학,0.531066,0.0,False
화공,0.385738,0.0,False
재료,0.332468,0.0,False
심리,0.744382,0.000774,False


In [17]:
# paper-research phase

phase_list = data['phase'].unique()
result = {}
for i in phase_list:
    pub_data = data[data['pub_nm'].notnull()]
    pub_normality = stats.shapiro(pub_data.loc[pub_data['phase'] == i, 'pub_nm'])
    result[i] = [pub_normality.statistic, pub_normality.pvalue, pub_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
기타,0.731177,0.0,False
개발연구,0.031299,0.0,False
응용연구,0.206633,0.0,False
기초연구,0.432092,0.0,False


In [18]:
# paper-technology type

technology_type_list = data['technology_type'].unique()
result={}

for i in technology_type_list:
    if len(pub_data[pub_data['technology_type'] == i]) > 3:
        pub_data = data[data['pub_nm'].notnull()]
        pub_normality = stats.shapiro(pub_data.loc[pub_data['technology_type'] == i, 'pub_nm'])
        result[i] = [pub_normality.statistic, pub_normality.pvalue, pub_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
에너지/자원,0.473411,0.0,False
원자력,0.322023,0.0,False
전기/전자,0.553284,0.0,False
보건의료,0.087002,0.0,False
생명과학,0.535011,0.0,False
뇌과학,0.511844,0.0,False
화학,0.619913,0.0,False
화공,0.537028,0.0,False
재료,0.581003,0.0,False
심리,0.801739,0.000035,False


In [19]:
# royalty-research phase

phase_list = data['phase'].unique()
result = {}
for i in phase_list:
    royalty_data = data[data['royalty_nm'].notnull()]
    royalty_normality = stats.shapiro(royalty_data.loc[royalty_data['phase'] == i, 'royalty_nm'])
    result[i] = [royalty_normality.statistic, royalty_normality.pvalue, royalty_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
기타,0.148401,0.0,False
개발연구,0.138077,0.0,False
응용연구,0.15579,0.0,False
기초연구,0.250723,0.0,False


In [20]:
# royalty-technology type

technology_type_list = data['technology_type'].unique()
result={}

for i in technology_type_list:
    if len(royalty_data[royalty_data['technology_type'] == i]) > 3:
        royalty_data = data[data['royalty_nm'].notnull()]
        royalty_normality = stats.shapiro(royalty_data.loc[royalty_data['technology_type'] == i, 'royalty_nm'])
        result[i] = [royalty_normality.statistic, royalty_normality.pvalue, royalty_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
에너지/자원,0.521701,0.0,False
원자력,0.6631,0.0,False
전기/전자,0.490229,0.0,False
보건의료,0.236686,0.0,False
생명과학,0.355967,0.0,False
뇌과학,0.872802,0.00883,False
화학,0.080127,0.0,False
화공,0.463813,0.0,False
재료,0.111894,0.0,False
경제/경영,0.394098,0.0,False


In [21]:
# sales-research phase

phase_list = data['phase'].unique()
result = {}
for i in phase_list:
    sales_data = data[data['sales_nm'].notnull()]
    sales_normality = stats.shapiro(sales_data.loc[sales_data['phase'] == i, 'sales_nm'])
    result[i] = [sales_normality.statistic, sales_normality.pvalue, sales_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
기타,0.08312,0.0,False
개발연구,0.148533,0.0,False
응용연구,0.008728,0.0,False
기초연구,0.017454,0.0,False


In [22]:
# sales-technology type

technology_type_list = data['technology_type'].unique()
result={}

for i in technology_type_list:
    if len(sales_data[sales_data['technology_type'] == i]) > 3:
        sales_data = data[data['sales_nm'].notnull()]
        sales_normality = stats.shapiro(sales_data.loc[sales_data['technology_type'] == i, 'sales_nm'])
        result[i] = [sales_normality.statistic, sales_normality.pvalue, sales_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', '정규성 여부'])
result.T

,statistic,pvalue,정규성 여부
에너지/자원,0.257709,0.0,False
원자력,0.423952,0.0,False
전기/전자,0.110793,0.0,False
보건의료,0.168553,0.0,False
생명과학,0.132313,0.0,False
뇌과학,0.385746,0.0,False
화학,0.204974,0.0,False
화공,0.204628,0.0,False
재료,0.268617,0.0,False
심리,0.59416,0.00039,False


# Non-parametric ANOVA : Kruskal-Wallis test

## <font color=gray>Applied Patents</font>

In [23]:
app_data = data[data['app_nm'].notnull()]
result = stats.kruskal(app_data.loc[app_data['phase']=='기초연구', 'app_nm'],
                       app_data.loc[app_data['phase']=='응용연구', 'app_nm'],
                       app_data.loc[app_data['phase']=='개발연구', 'app_nm'],
                       app_data.loc[app_data['phase']=='기타', 'app_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=3494.43736728627, pvalue=0.0)
0.000000
Rejecting the null hypothesis : True


In [24]:
app_data = data[data['app_nm'].notnull()]
result = stats.kruskal(app_data.loc[app_data['technology_type']=='보건의료', 'app_nm'],
                       app_data.loc[app_data['technology_type']=='농림수산식품', 'app_nm'],
                       app_data.loc[app_data['technology_type']=='기계', 'app_nm'],
                       app_data.loc[app_data['technology_type']=='생명과학', 'app_nm'],
                       app_data.loc[app_data['technology_type']=='정보/통신', 'app_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=1237.8884343950222, pvalue=9.734070301315872e-267)
0.000000
Rejecting the null hypothesis : True


## <font color=gray>Registered Patents</font>

In [25]:
reg_data = data[data['reg_nm'].notnull()]
result = stats.kruskal(reg_data.loc[reg_data['phase']=='기초연구', 'reg_nm'],
                       reg_data.loc[reg_data['phase']=='응용연구', 'reg_nm'],
                       reg_data.loc[reg_data['phase']=='개발연구', 'reg_nm'],
                       reg_data.loc[reg_data['phase']=='기타', 'reg_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=608.608536993325, pvalue=1.371375193119815e-131)
0.000000
Rejecting the null hypothesis : True


In [26]:
reg_data = data[data['reg_nm'].notnull()]
result = stats.kruskal(reg_data.loc[reg_data['technology_type']=='보건의료', 'reg_nm'],
                       reg_data.loc[reg_data['technology_type']=='농림수산식품', 'reg_nm'],
                       reg_data.loc[reg_data['technology_type']=='기계', 'reg_nm'],
                       reg_data.loc[reg_data['technology_type']=='생명과학', 'reg_nm'],
                       reg_data.loc[reg_data['technology_type']=='정보/통신', 'reg_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=49.166362427091805, pvalue=5.390320624009894e-10)
0.000000
Rejecting the null hypothesis : True


## <font color=gray>Papers</font>

In [27]:
pub_data = data[data['pub_nm'].notnull()]
result = stats.kruskal(pub_data.loc[pub_data['phase']=='기초연구', 'pub_nm'],
                       pub_data.loc[pub_data['phase']=='응용연구', 'pub_nm'],
                       pub_data.loc[pub_data['phase']=='개발연구', 'pub_nm'],
                       pub_data.loc[pub_data['phase']=='기타', 'pub_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=8606.480680229086, pvalue=0.0)
0.000000
Rejecting the null hypothesis : True


In [28]:
pub_data = data[data['pub_nm'].notnull()]
result = stats.kruskal(pub_data.loc[pub_data['technology_type']=='보건의료', 'pub_nm'],
                       pub_data.loc[pub_data['technology_type']=='농림수산식품', 'pub_nm'],
                       pub_data.loc[pub_data['technology_type']=='기계', 'pub_nm'],
                       pub_data.loc[pub_data['technology_type']=='생명과학', 'pub_nm'],
                       pub_data.loc[pub_data['technology_type']=='정보/통신', 'pub_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=211.99761626649953, pvalue=9.878283352736941e-45)
0.000000
Rejecting the null hypothesis : True


## <font color=gray>Royalty</font>

In [29]:
royalty_data = data[data['royalty_nm'].notnull()]
result = stats.kruskal(royalty_data.loc[royalty_data['phase']=='기초연구', 'royalty_nm'],
                       royalty_data.loc[royalty_data['phase']=='응용연구', 'royalty_nm'],
                       royalty_data.loc[royalty_data['phase']=='개발연구', 'royalty_nm'],
                       royalty_data.loc[royalty_data['phase']=='기타', 'royalty_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=257.56470296125923, pvalue=1.5121506671055605e-55)
0.000000
Rejecting the null hypothesis : True


In [30]:
royalty_data = data[data['royalty_nm'].notnull()]
result = stats.kruskal(royalty_data.loc[royalty_data['technology_type']=='보건의료', 'royalty_nm'],
                       royalty_data.loc[royalty_data['technology_type']=='농림수산식품', 'royalty_nm'],
                       royalty_data.loc[royalty_data['technology_type']=='기계', 'royalty_nm'],
                       royalty_data.loc[royalty_data['technology_type']=='생명과학', 'royalty_nm'],
                       royalty_data.loc[royalty_data['technology_type']=='정보/통신', 'royalty_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=105.58619722685224, pvalue=6.352945812552518e-22)
0.000000
Rejecting the null hypothesis : True


## <font color=gray>Sales</font>

In [31]:
sales_data = data[data['sales_nm'].notnull()]
result = stats.kruskal(sales_data.loc[sales_data['phase']=='기초연구', 'sales_nm'],
                       sales_data.loc[sales_data['phase']=='응용연구', 'sales_nm'],
                       sales_data.loc[sales_data['phase']=='개발연구', 'sales_nm'],
                       sales_data.loc[sales_data['phase']=='기타', 'sales_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=1665.0498717928, pvalue=0.0)
0.000000
Rejecting the null hypothesis : True


In [32]:
sales_data = data[data['sales_nm'].notnull()]
result = stats.kruskal(sales_data.loc[sales_data['technology_type']=='보건의료', 'sales_nm'],
                       sales_data.loc[sales_data['technology_type']=='농림수산식품', 'sales_nm'],
                       sales_data.loc[sales_data['technology_type']=='기계', 'sales_nm'],
                       sales_data.loc[sales_data['technology_type']=='생명과학', 'sales_nm'],
                       sales_data.loc[sales_data['technology_type']=='정보/통신', 'sales_nm'])

print(result)
print(format(result.pvalue,'f'))
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=1560.797782277385, pvalue=0.0)
0.000000
Rejecting the null hypothesis : True


# Dunn's Test

## <font color=gray>Research phase-Applied patents</font>

In [33]:
# bonferroni

import scikit_posthocs as sp
pd.options.display.float_format = '{:.10f}'.format

p_values = sp.posthoc_dunn(a=app_data, val_col='app_nm', group_col='phase', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['개발연구'] = (p_values.loc['개발연구']>0) & (p_values.loc['개발연구']<0.05)
result['기초연구'] = (p_values.loc['기초연구']>0) & (p_values.loc['기초연구']<0.05)
result['기타'] = (p_values.loc['기타']>0) & (p_values.loc['기타']<0.05)
result['응용연구'] = (p_values.loc['응용연구']>0) & (p_values.loc['응용연구']<0.05)

result

,개발연구,기초연구,기타,응용연구
개발연구,False,False,True,True
기초연구,False,False,True,True
기타,True,True,False,True
응용연구,True,True,True,False


In [34]:
# holm
p_values = sp.posthoc_dunn(a=app_data, val_col='app_nm', group_col='phase', p_adjust = 'holm')
p_values

,개발연구,기초연구,기타,응용연구
개발연구,1.0000000000,0.0000000000,0.0000000000,0.0000000380
기초연구,0.0000000000,1.0000000000,0.0000000000,0.0000000000
기타,0.0000000000,0.0000000000,1.0000000000,0.0000000000
응용연구,0.0000000380,0.0000000000,0.0000000000,1.0000000000


## <font color=gray>Research phase-Registered patents</font>

In [35]:
# bonferroni
p_values = sp.posthoc_dunn(a=app_data, val_col='reg_nm', group_col='phase', p_adjust = 'bonferroni')
p_values

result = pd.DataFrame()
result['개발연구'] = (p_values.loc['개발연구']>0) & (p_values.loc['개발연구']<0.05)
result['기초연구'] = (p_values.loc['기초연구']>0) & (p_values.loc['기초연구']<0.05)
result['기타'] = (p_values.loc['기타']>0) & (p_values.loc['기타']<0.05)
result['응용연구'] = (p_values.loc['응용연구']>0) & (p_values.loc['응용연구']<0.05)

result

,개발연구,기초연구,기타,응용연구
개발연구,False,True,True,True
기초연구,True,False,True,True
기타,True,True,False,True
응용연구,True,True,True,False


In [36]:
# holm
p_values = sp.posthoc_dunn(a=app_data, val_col='reg_nm', group_col='phase', p_adjust = 'holm')
p_values

,개발연구,기초연구,기타,응용연구
개발연구,1.0000000000,0.0000000000,0.0000000000,0.0009715203
기초연구,0.0000000000,1.0000000000,0.0077853916,0.0000000000
기타,0.0000000000,0.0077853916,1.0000000000,0.0000000000
응용연구,0.0009715203,0.0000000000,0.0000000000,1.0000000000


## <font color=gray>Research phase-Papers</font>

In [37]:
# bonferroni
p_values = sp.posthoc_dunn(a=app_data, val_col='pub_nm', group_col='phase', p_adjust = 'bonferroni')
p_values

result = pd.DataFrame()
result['개발연구'] = (p_values.loc['개발연구']>0) & (p_values.loc['개발연구']<0.05)
result['기초연구'] = (p_values.loc['기초연구']>0) & (p_values.loc['기초연구']<0.05)
result['기타'] = (p_values.loc['기타']>0) & (p_values.loc['기타']<0.05)
result['응용연구'] = (p_values.loc['응용연구']>0) & (p_values.loc['응용연구']<0.05)

result

,개발연구,기초연구,기타,응용연구
개발연구,False,True,True,True
기초연구,True,False,True,True
기타,True,True,False,True
응용연구,True,True,True,False


In [38]:
# holm
p_values = sp.posthoc_dunn(a=app_data, val_col='pub_nm', group_col='phase', p_adjust = 'holm')
p_values

,개발연구,기초연구,기타,응용연구
개발연구,1.0000000000,0.0000000000,0.0000000000,0.0000000000
기초연구,0.0000000000,1.0000000000,0.0000000000,0.0000000000
기타,0.0000000000,0.0000000000,1.0000000000,0.0000000000
응용연구,0.0000000000,0.0000000000,0.0000000000,1.0000000000


## <font color=gray>Research phase-Royalty</font>

In [39]:
# bonferroni
p_values = sp.posthoc_dunn(a=app_data, val_col='royalty_nm', group_col='phase', p_adjust = 'bonferroni')
p_values

result = pd.DataFrame()
result['개발연구'] = (p_values.loc['개발연구']>0) & (p_values.loc['개발연구']<0.05)
result['기초연구'] = (p_values.loc['기초연구']>0) & (p_values.loc['기초연구']<0.05)
result['기타'] = (p_values.loc['기타']>0) & (p_values.loc['기타']<0.05)
result['응용연구'] = (p_values.loc['응용연구']>0) & (p_values.loc['응용연구']<0.05)

result

,개발연구,기초연구,기타,응용연구
개발연구,False,False,False,False
기초연구,False,False,False,False
기타,False,False,False,False
응용연구,False,False,False,False


In [40]:
# holm
p_values = sp.posthoc_dunn(a=app_data, val_col='royalty_nm', group_col='phase', p_adjust = 'holm')
p_values

,개발연구,기초연구,기타,응용연구
개발연구,1.0000000000,1.0000000000,0.8464317098,1.0000000000
기초연구,1.0000000000,1.0000000000,1.0000000000,1.0000000000
기타,0.8464317098,1.0000000000,1.0000000000,1.0000000000
응용연구,1.0000000000,1.0000000000,1.0000000000,1.0000000000


## <font color=gray>Research phase-Sales</font>

In [41]:
# bonferroni
p_values = sp.posthoc_dunn(a=app_data, val_col='sales_nm', group_col='phase', p_adjust = 'bonferroni')
p_values

result = pd.DataFrame()
result['개발연구'] = (p_values.loc['개발연구']>0) & (p_values.loc['개발연구']<0.05)
result['기초연구'] = (p_values.loc['기초연구']>0) & (p_values.loc['기초연구']<0.05)
result['기타'] = (p_values.loc['기타']>0) & (p_values.loc['기타']<0.05)
result['응용연구'] = (p_values.loc['응용연구']>0) & (p_values.loc['응용연구']<0.05)

result

,개발연구,기초연구,기타,응용연구
개발연구,False,False,False,False
기초연구,False,False,False,False
기타,False,False,False,False
응용연구,False,False,False,False


In [42]:
# holm
p_values = sp.posthoc_dunn(a=app_data, val_col='sales_nm', group_col='phase', p_adjust = 'holm')
p_values

,개발연구,기초연구,기타,응용연구
개발연구,1.0000000000,0.4658750309,1.0000000000,0.1718654497
기초연구,0.4658750309,1.0000000000,1.0000000000,1.0000000000
기타,1.0000000000,1.0000000000,1.0000000000,1.0000000000
응용연구,0.1718654497,1.0000000000,1.0000000000,1.0000000000


## <font color=gray>Technology type-Applied patents</font>

In [43]:
# bonferroni
app_data = app_data[(app_data['technology_type']=='보건의료') | (app_data['technology_type']=='농림수산식품') | (app_data['technology_type']=='기계') | (app_data['technology_type']=='생명과학') | (app_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=app_data, val_col='app_nm', group_col='technology_type', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['기계'] = (p_values.loc['기계']>0) & (p_values.loc['기계']<0.05)
result['농림수산식품'] = (p_values.loc['농림수산식품']>0) & (p_values.loc['농림수산식품']<0.05)
result['보건의료'] = (p_values.loc['보건의료']>0) & (p_values.loc['보건의료']<0.05)
result['생명과학'] = (p_values.loc['생명과학']>0) & (p_values.loc['생명과학']<0.05)
result['정보/통신'] = (p_values.loc['정보/통신']>0) & (p_values.loc['정보/통신']<0.05)
result

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,False,True,True,True,True
농림수산식품,True,False,True,True,True
보건의료,True,True,False,True,False
생명과학,True,True,True,False,True
정보/통신,True,True,False,True,False


In [44]:
# holm
app_data = app_data[(app_data['technology_type']=='보건의료') | (app_data['technology_type']=='농림수산식품') | (app_data['technology_type']=='기계') | (app_data['technology_type']=='생명과학') | (app_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=app_data, val_col='app_nm', group_col='technology_type', p_adjust = 'holm')
p_values

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
농림수산식품,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000
보건의료,0.0000000000,0.0000000000,1.0000000000,0.0000128314,0.0201601533
생명과학,0.0000000000,0.0000000000,0.0000128314,1.0000000000,0.0000000005
정보/통신,0.0000000000,0.0000000000,0.0201601533,0.0000000005,1.0000000000


## <font color=gray>Technology type-Registered patents</font>

In [45]:
# bonferroni
reg_data = reg_data[(reg_data['technology_type']=='보건의료') | (reg_data['technology_type']=='농림수산식품') | (reg_data['technology_type']=='기계') | (reg_data['technology_type']=='생명과학') | (reg_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=reg_data, val_col='reg_nm', group_col='technology_type', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['기계'] = (p_values.loc['기계']>0) & (p_values.loc['기계']<0.05)
result['농림수산식품'] = (p_values.loc['농림수산식품']>0) & (p_values.loc['농림수산식품']<0.05)
result['보건의료'] = (p_values.loc['보건의료']>0) & (p_values.loc['보건의료']<0.05)
result['생명과학'] = (p_values.loc['생명과학']>0) & (p_values.loc['생명과학']<0.05)
result['정보/통신'] = (p_values.loc['정보/통신']>0) & (p_values.loc['정보/통신']<0.05)
result

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,False,False,True,False,False
농림수산식품,False,False,True,False,False
보건의료,True,True,False,True,True
생명과학,False,False,True,False,False
정보/통신,False,False,True,False,False


In [46]:
# holm
reg_data = reg_data[(reg_data['technology_type']=='보건의료') | (reg_data['technology_type']=='농림수산식품') | (reg_data['technology_type']=='기계') | (reg_data['technology_type']=='생명과학') | (reg_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=reg_data, val_col='reg_nm', group_col='technology_type', p_adjust = 'holm')
p_values

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,1.0000000000,0.4775456523,0.0000043084,0.5165160847,0.6735698991
농림수산식품,0.4775456523,1.0000000000,0.0000000013,0.0376844084,0.2264824825
보건의료,0.0000043084,0.0000000013,1.0000000000,0.0143858349,0.0000361361
생명과학,0.5165160847,0.0376844084,0.0143858349,1.0000000000,0.6735698991
정보/통신,0.6735698991,0.2264824825,0.0000361361,0.6735698991,1.0000000000


## <font color=gray>Technology type-Papers</font>

In [47]:
# bonferroni
pub_data = pub_data[(pub_data['technology_type']=='보건의료') | (pub_data['technology_type']=='농림수산식품') | (pub_data['technology_type']=='기계') | (pub_data['technology_type']=='생명과학') | (pub_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=pub_data, val_col='pub_nm', group_col='technology_type', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['기계'] = (p_values.loc['기계']>0) & (p_values.loc['기계']<0.05)
result['농림수산식품'] = (p_values.loc['농림수산식품']>0) & (p_values.loc['농림수산식품']<0.05)
result['보건의료'] = (p_values.loc['보건의료']>0) & (p_values.loc['보건의료']<0.05)
result['생명과학'] = (p_values.loc['생명과학']>0) & (p_values.loc['생명과학']<0.05)
result['정보/통신'] = (p_values.loc['정보/통신']>0) & (p_values.loc['정보/통신']<0.05)
result

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,False,True,True,True,False
농림수산식품,True,False,True,True,False
보건의료,True,True,False,False,True
생명과학,True,True,False,False,True
정보/통신,False,False,True,True,False


In [48]:
# holm
pub_data = pub_data[(pub_data['technology_type']=='보건의료') | (pub_data['technology_type']=='농림수산식품') | (pub_data['technology_type']=='기계') | (pub_data['technology_type']=='생명과학') | (pub_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=pub_data, val_col='pub_nm', group_col='technology_type', p_adjust = 'holm')
p_values

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,1.0000000000,0.0141702304,0.0000000000,0.0000000000,0.0340311409
농림수산식품,0.0141702304,1.0000000000,0.0000000000,0.0000000000,1.0000000000
보건의료,0.0000000000,0.0000000000,1.0000000000,1.0000000000,0.0000000000
생명과학,0.0000000000,0.0000000000,1.0000000000,1.0000000000,0.0000000000
정보/통신,0.0340311409,1.0000000000,0.0000000000,0.0000000000,1.0000000000


## <font color=gray>Technology type-Royalty</font>

In [49]:
# bonferroni
royalty_data = royalty_data[(royalty_data['technology_type']=='보건의료') | (royalty_data['technology_type']=='농림수산식품') | (royalty_data['technology_type']=='기계') | (royalty_data['technology_type']=='생명과학') | (royalty_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=royalty_data, val_col='royalty_nm', group_col='technology_type', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['기계'] = (p_values.loc['기계']>0) & (p_values.loc['기계']<0.05)
result['농림수산식품'] = (p_values.loc['농림수산식품']>0) & (p_values.loc['농림수산식품']<0.05)
result['보건의료'] = (p_values.loc['보건의료']>0) & (p_values.loc['보건의료']<0.05)
result['생명과학'] = (p_values.loc['생명과학']>0) & (p_values.loc['생명과학']<0.05)
result['정보/통신'] = (p_values.loc['정보/통신']>0) & (p_values.loc['정보/통신']<0.05)
result

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,False,True,True,True,True
농림수산식품,True,False,True,True,False
보건의료,True,True,False,False,True
생명과학,True,True,False,False,True
정보/통신,True,False,True,True,False


In [50]:
# holm
royalty_data = royalty_data[(royalty_data['technology_type']=='보건의료') | (royalty_data['technology_type']=='농림수산식품') | (royalty_data['technology_type']=='기계') | (royalty_data['technology_type']=='생명과학') | (royalty_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=royalty_data, val_col='royalty_nm', group_col='technology_type', p_adjust = 'holm')
p_values

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,1.0000000000,0.0006065210,0.0000087226,0.0021776909,0.0021945323
농림수산식품,0.0006065210,1.0000000000,0.0000000000,0.0000000016,0.9727452316
보건의료,0.0000087226,0.0000000000,1.0000000000,0.9727452316,0.0000000000
생명과학,0.0021776909,0.0000000016,0.9727452316,1.0000000000,0.0000000113
정보/통신,0.0021945323,0.9727452316,0.0000000000,0.0000000113,1.0000000000


## <font color=gray>Technology type-Sales</font>

In [51]:
# bonferroni
sales_data = sales_data[(sales_data['technology_type']=='보건의료') | (sales_data['technology_type']=='농림수산식품') | (sales_data['technology_type']=='기계') | (sales_data['technology_type']=='생명과학') | (sales_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=sales_data, val_col='sales_nm', group_col='technology_type', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['기계'] = (p_values.loc['기계']>0) & (p_values.loc['기계']<0.05)
result['농림수산식품'] = (p_values.loc['농림수산식품']>0) & (p_values.loc['농림수산식품']<0.05)
result['보건의료'] = (p_values.loc['보건의료']>0) & (p_values.loc['보건의료']<0.05)
result['생명과학'] = (p_values.loc['생명과학']>0) & (p_values.loc['생명과학']<0.05)
result['정보/통신'] = (p_values.loc['정보/통신']>0) & (p_values.loc['정보/통신']<0.05)
result

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,False,True,True,False,False
농림수산식품,True,False,True,True,True
보건의료,True,True,False,True,True
생명과학,False,True,True,False,True
정보/통신,False,True,True,True,False


In [52]:
# holm
sales_data = sales_data[(sales_data['technology_type']=='보건의료') | (sales_data['technology_type']=='농림수산식품') | (sales_data['technology_type']=='기계') | (sales_data['technology_type']=='생명과학') | (sales_data['technology_type']=='정보/통신')]
p_values = sp.posthoc_dunn(a=sales_data, val_col='sales_nm', group_col='technology_type', p_adjust = 'holm')
p_values

,기계,농림수산식품,보건의료,생명과학,정보/통신
기계,1.0000000000,0.0000000000,0.0000000000,0.0217900184,0.1318872622
농림수산식품,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000
보건의료,0.0000000000,0.0000000000,1.0000000000,0.0000000554,0.0000000000
생명과학,0.0217900184,0.0000000000,0.0000000554,1.0000000000,0.0015337014
정보/통신,0.1318872622,0.0000000000,0.0000000000,0.0015337014,1.0000000000


# Performance analysis by management institution

In [53]:
pd.reset_option('display.float_format')

In [54]:
# target projects under the MSIT
dataset = data[(data['ministry']=='과학기술정보통신부') | (data['ministry']=='미래창조과학부')]
dataset

,year,project_name,division,id_prev,id,type_institution,institution,start_date,end_date,technology_type,...,royalty,sales,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm,pat_nm,start_year,end_year
4235,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711028873,1711042878,대학,서울대학교,2015-09-01,2022-08-31,생명과학,...,NaN,NaN,0.800000,0.800000,NaN,NaN,NaN,1.000000,2015,2022.0
4236,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711032072,1711044760,출연연구소,한국생명공학연구원,2012-11-30,2017-11-29,생명과학,...,NaN,NaN,NaN,NaN,4.166667,NaN,NaN,NaN,2012,2017.0
4237,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711030723,1711042978,출연연구소,한국전자통신연구원,2014-10-01,2022-09-30,생명과학,...,NaN,NaN,0.332815,0.000000,0.665631,NaN,NaN,0.000000,2014,2022.0
4238,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711030789,1711044431,대학,연세대학교,2014-11-01,2022-10-31,생명과학,...,NaN,NaN,NaN,NaN,1.199998,NaN,NaN,NaN,2014,2022.0
4239,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711029548,1711043431,출연연구소,한국생명공학연구원,2014-10-01,2022-09-30,생명과학,...,NaN,NaN,1.028570,0.342857,0.685714,NaN,NaN,0.333333,2014,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98220,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711099086,대학,한양대학교(ERICA캠퍼스),2019-09-27,2022-12-31,재료,...,NaN,NaN,2.061856,0.000000,NaN,NaN,NaN,0.000000,2019,2022.0
98221,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711098190,대학,포항공과대학교,2019-07-15,2025-06-30,재료,...,NaN,NaN,0.400000,0.000000,0.200000,NaN,NaN,0.000000,2019,2025.0
98222,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711077275,1711093328,기타,전자부품연구원,2018-07-16,2024-07-15,전기/전자,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2018,2024.0
98223,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711076433,1711083081,대학,고려대학교,2018-07-16,2024-07-15,재료,...,NaN,NaN,3.428571,0.000000,5.142857,NaN,NaN,0.000000,2018,2024.0


In [55]:
dataset.loc[dataset['management_institution'] == '산업기술연구회', 'management_institution'] = '국가과학기술연구회'
dataset.loc[dataset['management_institution'] == '정보통신기술진흥센터', 'management_institution'] = '정보통신기획평가원'
dataset.loc[dataset['management_institution'] == '미래창조과학부', 'management_institution'] = '과학기술정보통신부'

In [56]:
freq_table = pd.Series(dataset['management_institution']).value_counts()
freq_table = pd.DataFrame(freq_table)
freq_table

,management_institution
한국연구재단,28126
정보통신기획평가원,3163
국가과학기술연구회,3146
과학기술정보통신부,1069
한국산업기술평가관리원,677
국토교통과학기술진흥원,2


In [57]:
freq_table = pd.DataFrame()
management_institution = dataset['management_institution'].unique()
phase = dataset['phase'].unique()

for i in management_institution:
    for j in phase:
        freq_table.loc[i,j] = dataset[(dataset['management_institution']==i) & (dataset['phase']==j)]['id'].count()

freq_table['합계'] = freq_table['기초연구']+freq_table['응용연구']+freq_table['개발연구']+freq_table['기타']
freq_table

,기초연구,개발연구,응용연구,기타,합계
한국연구재단,24092.0,1955.0,1900.0,179.0,28126.0
과학기술정보통신부,383.0,407.0,179.0,100.0,1069.0
국가과학기술연구회,1178.0,993.0,660.0,315.0,3146.0
한국산업기술평가관리원,215.0,296.0,138.0,28.0,677.0
정보통신기획평가원,782.0,1219.0,716.0,446.0,3163.0
국토교통과학기술진흥원,0.0,2.0,0.0,0.0,2.0


In [58]:
freq_table_ratio = pd.DataFrame()
freq_table_ratio['기초연구'] = freq_table['기초연구']/freq_table['합계']
freq_table_ratio['응용연구'] = freq_table['응용연구']/freq_table['합계']
freq_table_ratio['개발연구'] = freq_table['개발연구']/freq_table['합계']
freq_table_ratio['기타'] = freq_table['기타']/freq_table['합계']

freq_table_ratio

,기초연구,응용연구,개발연구,기타
한국연구재단,0.856574,0.067553,0.069509,0.006364
과학기술정보통신부,0.358279,0.167446,0.380730,0.093545
국가과학기술연구회,0.374444,0.209790,0.315639,0.100127
한국산업기술평가관리원,0.317578,0.203840,0.437223,0.041359
정보통신기획평가원,0.247234,0.226367,0.385394,0.141005
국토교통과학기술진흥원,0.000000,0.000000,1.000000,0.000000


In [59]:
performance_table = pd.DataFrame()

for i in management_institution:
    for j in ['app_nm', 'reg_nm', 'pub_nm', 'royalty_nm', 'sales_nm']:
        performance_table.loc[i,j] = dataset[dataset['management_institution']==i][j].mean()

performance_table

,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm
한국연구재단,1.968049,0.611811,3.701519,0.251580,9.396180
과학기술정보통신부,1.641336,0.653222,2.795882,0.129897,10.567962
국가과학기술연구회,0.821874,0.245155,0.764019,0.188586,0.630731
한국산업기술평가관리원,1.851559,0.520108,2.878473,0.206175,1.814195
정보통신기획평가원,1.115282,0.302919,0.648715,0.090512,2.709529
국토교통과학기술진흥원,0.239590,0.000000,NaN,NaN,NaN


In [60]:
performance_table = pd.DataFrame()

for i in management_institution:
    for j in ['app_nm', 'reg_nm', 'pub_nm', 'royalty_nm', 'sales_nm']:
        performance_table.loc[i,j] = dataset[dataset['management_institution']==i][j].std()

performance_table

,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm
한국연구재단,2.812945,1.268777,6.447037,1.113491,99.163952
과학기술정보통신부,3.574189,1.741417,5.109388,0.245419,70.195267
국가과학기술연구회,1.145378,0.562670,1.206460,0.322776,3.046482
한국산업기술평가관리원,1.903624,1.025743,3.506402,0.219017,4.782205
정보통신기획평가원,2.570576,0.716111,0.868678,0.128830,19.037465
국토교통과학기술진흥원,0.213236,0.000000,NaN,NaN,NaN


In [61]:
performance_table = pd.DataFrame()

for i in management_institution:
    for j in ['app_nm', 'reg_nm', 'pub_nm', 'royalty_nm', 'sales_nm']:
        performance_table.loc[i,j] = dataset[dataset['management_institution']==i][j].count()

performance_table

,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm
한국연구재단,12299.0,12299.0,24766.0,345.0,150.0
과학기술정보통신부,767.0,767.0,608.0,113.0,122.0
국가과학기술연구회,2396.0,2396.0,2050.0,1055.0,123.0
한국산업기술평가관리원,567.0,567.0,477.0,63.0,45.0
정보통신기획평가원,2936.0,2936.0,1271.0,928.0,797.0
국토교통과학기술진흥원,2.0,2.0,0.0,0.0,0.0


In [62]:
# 국토교통과학기술진흥원 제외
index = dataset[dataset['management_institution']=='국토교통과학기술진흥원'].index
dataset.drop(index, inplace=True)
dataset.reset_index()

,index,year,project_name,division,id_prev,id,type_institution,institution,start_date,end_date,...,royalty,sales,app_nm,reg_nm,pub_nm,royalty_nm,sales_nm,pat_nm,start_year,end_year
0,4235,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711028873,1711042878,대학,서울대학교,2015-09-01,2022-08-31,...,NaN,NaN,0.800000,0.800000,NaN,NaN,NaN,1.000000,2015,2022.0
1,4236,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711032072,1711044760,출연연구소,한국생명공학연구원,2012-11-30,2017-11-29,...,NaN,NaN,NaN,NaN,4.166667,NaN,NaN,NaN,2012,2017.0
2,4237,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711030723,1711042978,출연연구소,한국전자통신연구원,2014-10-01,2022-09-30,...,NaN,NaN,0.332815,0.000000,0.665631,NaN,NaN,0.000000,2014,2022.0
3,4238,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711030789,1711044431,대학,연세대학교,2014-11-01,2022-10-31,...,NaN,NaN,NaN,NaN,1.199998,NaN,NaN,NaN,2014,2022.0
4,4239,2016.0,포스트게놈신산업육성을위한다부처유전체(미래부),일반연구개발사업,1711029548,1711043431,출연연구소,한국생명공학연구원,2014-10-01,2022-09-30,...,NaN,NaN,1.028570,0.342857,0.685714,NaN,NaN,0.333333,2014,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36176,98220,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711099086,대학,한양대학교(ERICA캠퍼스),2019-09-27,2022-12-31,...,NaN,NaN,2.061856,0.000000,NaN,NaN,NaN,0.000000,2019,2022.0
36177,98221,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,NaN,1711098190,대학,포항공과대학교,2019-07-15,2025-06-30,...,NaN,NaN,0.400000,0.000000,0.200000,NaN,NaN,0.000000,2019,2025.0
36178,98222,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711077275,1711093328,기타,전자부품연구원,2018-07-16,2024-07-15,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2018,2024.0
36179,98223,2019.0,미래소재디스커버리지원(R&D),일반연구개발사업,1711076433,1711083081,대학,고려대학교,2018-07-16,2024-07-15,...,NaN,NaN,3.428571,0.000000,5.142857,NaN,NaN,0.000000,2018,2024.0


## Normality test

In [63]:
# Applied patent-management institution

management_institution = dataset['management_institution'].unique()
result = {}
for i in management_institution:
    if len(app_data[app_data['management_institution'] == i]) > 3:
        app_data = dataset[dataset['app_nm'].notnull()]
        app_normality = stats.shapiro(app_data.loc[app_data['management_institution'] == i, 'app_nm'])
        result[i] = [app_normality.statistic, app_normality.pvalue, app_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', 'normality test'])
result.T

,statistic,pvalue,normality test
한국연구재단,0.522444,0.0,False
과학기술정보통신부,0.402415,0.0,False
국가과학기술연구회,0.572182,0.0,False
한국산업기술평가관리원,0.775509,0.0,False
정보통신기획평가원,0.231174,0.0,False


In [64]:
# Registered patent-management institution

management_institution = dataset['management_institution'].unique()
result = {}
for i in management_institution:
    if len(reg_data[reg_data['management_institution'] == i]) > 3:
        reg_data = dataset[dataset['reg_nm'].notnull()]
        reg_normality = stats.shapiro(reg_data.loc[reg_data['management_institution'] == i, 'reg_nm'])
        result[i] = [reg_normality.statistic, reg_normality.pvalue, reg_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', 'normality test'])
result.T

,statistic,pvalue,normality test
한국연구재단,0.527387,0.0,False
과학기술정보통신부,0.393171,0.0,False
국가과학기술연구회,0.44299,0.0,False
한국산업기술평가관리원,0.582425,0.0,False
정보통신기획평가원,0.454166,0.0,False


In [65]:
# Papers-management institution

management_institution = dataset['management_institution'].unique()
result = {}
for i in management_institution:
    if len(pub_data[pub_data['management_institution'] == i]) > 3:
        pub_data = dataset[dataset['pub_nm'].notnull()]
        pub_normality = stats.shapiro(pub_data.loc[pub_data['management_institution'] == i, 'pub_nm'])
        result[i] = [pub_normality.statistic, pub_normality.pvalue, pub_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', 'normality test'])
result.T

,statistic,pvalue,normality test
한국연구재단,0.391482,0.0,False
과학기술정보통신부,0.541939,0.0,False
국가과학기술연구회,0.478053,0.0,False
한국산업기술평가관리원,0.734575,0.0,False
정보통신기획평가원,0.658035,0.0,False


In [66]:
# Royalty-management institution

management_institution = dataset['management_institution'].unique()
result = {}
for i in management_institution:
    if len(royalty_data[royalty_data['management_institution'] == i]) > 3:
        royalty_data = dataset[dataset['royalty_nm'].notnull()]
        royalty_normality = stats.shapiro(royalty_data.loc[royalty_data['management_institution'] == i, 'royalty_nm'])
        result[i] = [royalty_normality.statistic, royalty_normality.pvalue, royalty_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', 'normality test'])
result.T

,statistic,pvalue,normality test
한국연구재단,0.149205,0.0,False
과학기술정보통신부,0.519496,0.0,False
국가과학기술연구회,0.510095,0.0,False
한국산업기술평가관리원,0.819373,0.0,False
정보통신기획평가원,0.592742,0.0,False


In [67]:
# Sales-management institution

management_institution = dataset['management_institution'].unique()
result = {}
for i in management_institution:
    if len(sales_data[sales_data['management_institution'] == i]) > 3:
        sales_data = dataset[dataset['sales_nm'].notnull()]
        sales_normality = stats.shapiro(sales_data.loc[sales_data['management_institution'] == i, 'sales_nm'])
        result[i] = [sales_normality.statistic, sales_normality.pvalue, sales_normality.pvalue>0.05]

result = pd.DataFrame(result, index=['statistic', 'pvalue', 'normaltiy test'])
result.T

,statistic,pvalue,normaltiy test
한국연구재단,0.067307,0.0,False
과학기술정보통신부,0.130517,0.0,False
국가과학기술연구회,0.21673,0.0,False
한국산업기술평가관리원,0.399928,0.0,False
정보통신기획평가원,0.096491,0.0,False


## Non-Parametric ANOVA

In [68]:
app_data = dataset[dataset['app_nm'].notnull()]
result = stats.kruskal(app_data.loc[app_data['management_institution']=='한국연구재단', 'app_nm'],
                       app_data.loc[app_data['management_institution']=='과학기술정보통신부', 'app_nm'],
                       app_data.loc[app_data['management_institution']=='한국산업기술평가관리원', 'app_nm'],
                       app_data.loc[app_data['management_institution']=='국가과학기술연구회', 'app_nm'],
                       app_data.loc[app_data['management_institution']=='정보통신기획평가원', 'app_nm'])

print(result)
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=1804.5386308636512, pvalue=0.0)
Rejecting the null hypothesis : True


In [69]:
reg_data = dataset[dataset['reg_nm'].notnull()]
result = stats.kruskal(reg_data.loc[reg_data['management_institution']=='한국연구재단', 'reg_nm'],
                       reg_data.loc[reg_data['management_institution']=='과학기술정보통신부', 'reg_nm'],
                       reg_data.loc[reg_data['management_institution']=='한국산업기술평가관리원', 'reg_nm'],
                       reg_data.loc[reg_data['management_institution']=='국가과학기술연구회', 'reg_nm'],
                       reg_data.loc[reg_data['management_institution']=='정보통신기획평가원', 'reg_nm'])

print(result)
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=81.31363396081996, pvalue=9.176029811777317e-17)
Rejecting the null hypothesis : True


In [70]:
pub_data = dataset[dataset['pub_nm'].notnull()]
result = stats.kruskal(pub_data.loc[pub_data['management_institution']=='한국연구재단', 'pub_nm'],
                       pub_data.loc[pub_data['management_institution']=='과학기술정보통신부', 'pub_nm'],
                       pub_data.loc[pub_data['management_institution']=='한국산업기술평가관리원', 'pub_nm'],
                       pub_data.loc[pub_data['management_institution']=='국가과학기술연구회', 'pub_nm'],
                       pub_data.loc[pub_data['management_institution']=='정보통신기획평가원', 'pub_nm'])

print(result)
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=4854.7754534494825, pvalue=0.0)
Rejecting the null hypothesis : True


In [71]:
royalty_data = dataset[dataset['royalty_nm'].notnull()]
result = stats.kruskal(royalty_data.loc[royalty_data['management_institution']=='한국연구재단', 'royalty_nm'],
                       royalty_data.loc[royalty_data['management_institution']=='과학기술정보통신부', 'royalty_nm'],
                       royalty_data.loc[royalty_data['management_institution']=='한국산업기술평가관리원', 'royalty_nm'],
                       royalty_data.loc[royalty_data['management_institution']=='국가과학기술연구회', 'royalty_nm'],
                       royalty_data.loc[royalty_data['management_institution']=='정보통신기획평가원', 'royalty_nm'])

print(result)
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=88.27494194824419, pvalue=3.0610972903994344e-18)
Rejecting the null hypothesis : True


In [72]:
sales_data = dataset[dataset['sales_nm'].notnull()]
result = stats.kruskal(sales_data.loc[sales_data['management_institution']=='한국연구재단', 'sales_nm'],
                       sales_data.loc[sales_data['management_institution']=='과학기술정보통신부', 'sales_nm'],
                       sales_data.loc[sales_data['management_institution']=='한국산업기술평가관리원', 'sales_nm'],
                       sales_data.loc[sales_data['management_institution']=='국가과학기술연구회', 'sales_nm'],
                       sales_data.loc[sales_data['management_institution']=='정보통신기획평가원', 'sales_nm'])

print(result)
print('Rejecting the null hypothesis :', result.pvalue<0.05)

KruskalResult(statistic=205.2165605714637, pvalue=2.8391251677131624e-43)
Rejecting the null hypothesis : True


## Post-hoc Test(Dunn's Test)

In [73]:
# bonferroni

import scikit_posthocs as sp

p_values = sp.posthoc_dunn(a=app_data, val_col='app_nm', group_col='management_institution', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['과학기술정보통신부'] = (p_values.loc['과학기술정보통신부']>0) & (p_values.loc['과학기술정보통신부']<0.05)
result['국가과학기술연구회'] = (p_values.loc['국가과학기술연구회']>0) & (p_values.loc['국가과학기술연구회']<0.05)
result['정보통신기획평가원'] = (p_values.loc['정보통신기획평가원']>0) & (p_values.loc['정보통신기획평가원']<0.05)
result['한국산업기술평가관리원'] = (p_values.loc['한국산업기술평가관리원']>0) & (p_values.loc['한국산업기술평가관리원']<0.05)
result['한국연구재단'] = (p_values.loc['한국연구재단']>0) & (p_values.loc['한국연구재단']<0.05)

result

,과학기술정보통신부,국가과학기술연구회,정보통신기획평가원,한국산업기술평가관리원,한국연구재단
과학기술정보통신부,False,True,False,True,True
국가과학기술연구회,True,False,True,True,True
정보통신기획평가원,False,True,False,True,True
한국산업기술평가관리원,True,True,True,False,False
한국연구재단,True,True,True,False,False


In [74]:
# bonferroni

p_values = sp.posthoc_dunn(a=reg_data, val_col='reg_nm', group_col='management_institution', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['과학기술정보통신부'] = (p_values.loc['과학기술정보통신부']>0) & (p_values.loc['과학기술정보통신부']<0.05)
result['국가과학기술연구회'] = (p_values.loc['국가과학기술연구회']>0) & (p_values.loc['국가과학기술연구회']<0.05)
result['정보통신기획평가원'] = (p_values.loc['정보통신기획평가원']>0) & (p_values.loc['정보통신기획평가원']<0.05)
result['한국산업기술평가관리원'] = (p_values.loc['한국산업기술평가관리원']>0) & (p_values.loc['한국산업기술평가관리원']<0.05)
result['한국연구재단'] = (p_values.loc['한국연구재단']>0) & (p_values.loc['한국연구재단']<0.05)

result

,과학기술정보통신부,국가과학기술연구회,정보통신기획평가원,한국산업기술평가관리원,한국연구재단
과학기술정보통신부,False,True,True,True,True
국가과학기술연구회,True,False,False,False,True
정보통신기획평가원,True,False,False,False,True
한국산업기술평가관리원,True,False,False,False,False
한국연구재단,True,True,True,False,False


In [75]:
# bonferroni

p_values = sp.posthoc_dunn(a=pub_data, val_col='pub_nm', group_col='management_institution', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['과학기술정보통신부'] = (p_values.loc['과학기술정보통신부']>0) & (p_values.loc['과학기술정보통신부']<0.05)
result['국가과학기술연구회'] = (p_values.loc['국가과학기술연구회']>0) & (p_values.loc['국가과학기술연구회']<0.05)
result['정보통신기획평가원'] = (p_values.loc['정보통신기획평가원']>0) & (p_values.loc['정보통신기획평가원']<0.05)
result['한국산업기술평가관리원'] = (p_values.loc['한국산업기술평가관리원']>0) & (p_values.loc['한국산업기술평가관리원']<0.05)
result['한국연구재단'] = (p_values.loc['한국연구재단']>0) & (p_values.loc['한국연구재단']<0.05)

result

,과학기술정보통신부,국가과학기술연구회,정보통신기획평가원,한국산업기술평가관리원,한국연구재단
과학기술정보통신부,False,True,True,True,True
국가과학기술연구회,True,False,False,True,False
정보통신기획평가원,True,False,False,True,False
한국산업기술평가관리원,True,True,True,False,True
한국연구재단,True,False,False,True,False


In [76]:
# bonferroni

p_values = sp.posthoc_dunn(a=royalty_data, val_col='royalty_nm', group_col='management_institution', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['과학기술정보통신부'] = (p_values.loc['과학기술정보통신부']>0) & (p_values.loc['과학기술정보통신부']<0.05)
result['국가과학기술연구회'] = (p_values.loc['국가과학기술연구회']>0) & (p_values.loc['국가과학기술연구회']<0.05)
result['정보통신기획평가원'] = (p_values.loc['정보통신기획평가원']>0) & (p_values.loc['정보통신기획평가원']<0.05)
result['한국산업기술평가관리원'] = (p_values.loc['한국산업기술평가관리원']>0) & (p_values.loc['한국산업기술평가관리원']<0.05)
result['한국연구재단'] = (p_values.loc['한국연구재단']>0) & (p_values.loc['한국연구재단']<0.05)

result

,과학기술정보통신부,국가과학기술연구회,정보통신기획평가원,한국산업기술평가관리원,한국연구재단
과학기술정보통신부,False,True,False,True,True
국가과학기술연구회,True,False,True,False,False
정보통신기획평가원,False,True,False,True,True
한국산업기술평가관리원,True,False,True,False,False
한국연구재단,True,False,True,False,False


In [77]:
# bonferroni

p_values = sp.posthoc_dunn(a=sales_data, val_col='sales_nm', group_col='management_institution', p_adjust = 'bonferroni')

result = pd.DataFrame()
result['과학기술정보통신부'] = (p_values.loc['과학기술정보통신부']>0) & (p_values.loc['과학기술정보통신부']<0.05)
result['국가과학기술연구회'] = (p_values.loc['국가과학기술연구회']>0) & (p_values.loc['국가과학기술연구회']<0.05)
result['정보통신기획평가원'] = (p_values.loc['정보통신기획평가원']>0) & (p_values.loc['정보통신기획평가원']<0.05)
result['한국산업기술평가관리원'] = (p_values.loc['한국산업기술평가관리원']>0) & (p_values.loc['한국산업기술평가관리원']<0.05)
result['한국연구재단'] = (p_values.loc['한국연구재단']>0) & (p_values.loc['한국연구재단']<0.05)

result

,과학기술정보통신부,국가과학기술연구회,정보통신기획평가원,한국산업기술평가관리원,한국연구재단
과학기술정보통신부,False,True,False,False,True
국가과학기술연구회,True,False,True,True,True
정보통신기획평가원,False,True,False,False,True
한국산업기술평가관리원,False,True,False,False,True
한국연구재단,True,True,True,True,False
